In [1]:
from textblob import TextBlob 
from datetime import datetime
from pythainlp import word_tokenize
from nltk.corpus import stopwords
import nltk
import langdetect
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
import requests

In [2]:
filename = glob.glob('*.csv')

In [52]:
nltk.download('stopwords')  
class DataManager:
    def __init__(self):
        ##-------------------- twitter --------------------##
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self._api = tw.API(auth, wait_on_rate_limit=True)

        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

        self.keys = []
        self.df = None
        self._start = 0
        self.filenames = []

    def getSentimentENG(self,text):
        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def getSentimentTH(self,text):
        text = re.sub(r'[%]',' ',text)
        params = {'text':text}
        response = requests.get(self._url, headers=self._headers, params=params)
        try:
            polarity = str(response.json()['sentiment']['polarity'])
        except (KeyError):
            polarity = 'neutral'
        return polarity

    def formatdatetime(self,column):
        self.df[column] = pd.to_datetime(self.df[column]).dt.strftime('%Y/%m/%d') #dmY ทีหลัง
        self.df[column] = pd.to_datetime(self.df[column])
    
    def sortdf(self,columns):
        self.df.sort_values(by=columns,inplace=True)
        return self.df

    def unionfile(self,filenames):              #type filename -> list
        self._start = 0
        self.filenames = filenames
        for file in filenames:
            df1 = pd.read_csv(file)
            if self._start != 0:
                self.df = pd.concat([self.df,df1])
                self.df.drop_duplicates(keep='last',inplace=True)
            else:
                self.df = df1
                self._start += 1
        return self.df
    
    def setnewDF(self,dataframe):
        self.df = dataframe
        return self.df
    
    def setdefaultDF(self):
        self.df = self.unionfile(self.filenames)
        return self.df

    # def getperiod(self,since,until):  ####column for twitter
    #     self.formatdatetime('Time')
    #     self.df.sort_values(by=['Time','Keyword'],inplace=True)
    #     if since == None and until != None:
    #         mask = (self.df['Time']<=until)
    #     elif since != None and until == None:
    #         mask = (self.df['Time']>=since)
    #     elif since != None and until != None:
    #         mask = (self.df['Time']>=since) & (self.df['Time']<=until)
    #     else:
    #         return
    #     #self.df = self.df.loc[mask]
    #     return self.df.loc[mask]
    
    def getperiod(self,since,until):  ####column for twitter
        self.formatdatetime('Time')
        dff = self.df
        dff.sort_values(by=['Time','Keyword'],inplace=True)
        if since == None and until != None:
            mask = (dff['Time']<=until)
        elif since != None and until == None:
            mask = (dff['Time']>=since)
        elif since != None and until != None:
            mask = (dff['Time']>=since) & (dff['Time']<=until)
        else:
            return
        return dff.loc[mask]

    # def getrowwithkeys(self,keys):              #type keys -> list
    #     df = self.df
    #     self.df = df.loc[df['Keyword'].isin(keys)]
    #     self.df = self.df.loc[self.df['Keyword'].isin(keys)]
    #     return self.df
    
    def getrowwithkeys(self,keys):              #type keys -> list
        df = self.df
        return df.loc[df['Keyword'].isin(keys)]

    def collectword(self):            #before edit collect thai can not use
        dataframe = self.df
        en_stops = set(stopwords.words('english'))
        word = {}
        for i in dataframe['Tweet']:    #only tweet
            if langdetect.detect(i) != 'th':
                allwords = i.split()
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            else:
                allwords = word_tokenize(i, engine='newmm')
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
        del word['RT']
        del word[' ']   #for thai language
        sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
        return sortword     #tuple in list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
dm = DataManager()
dff = dm.unionfile(filename)
#dm.unionfile(['tweet_data_2732022.csv'])
dff

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
0,animation,dialhforhero,0.0,0.0,An animation trailer x100 times better than mo...,2022-03-19 11:44:27+00:00,NaN,NaN,NaN,NaN,NaN
1,animation,MaFt,0.0,0.0,@winstano My uncle @Billy_Allison did the anim...,2022-03-19 09:09:22+00:00,NaN,NaN,NaN,NaN,NaN
2,animation,plushladyy,0.0,0.0,Ghhh there was this animation about moon danci...,2022-03-19 09:39:41+00:00,NaN,NaN,NaN,NaN,NaN
3,animation,aeyvindr,0.0,0.0,I wonder how would the past me say seeing my d...,2022-03-19 12:14:58+00:00,NaN,NaN,NaN,NaN,NaN
4,animation,zeze_luva,0.0,0.0,"The plot of ""I’ve Gone Viral Thanks to My Crus...",2022-03-19 15:58:17+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4898,อนิเมะแนะนำ,Worra_K,41.0,0.0,RT bbrinkk มีกิจกรรมฟินๆมาแนะนำ สาววายถูกใจสิ่...,2022-03-24 10:49:48+00:00,th,NaN,['สาววายถ'],positive,97.30
4899,อนิเมะแนะนำ,W3NY7,33.0,0.0,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,2022-03-24 08:41:09+00:00,th,18 up /,['แนะนำอ'],positive,66.67
4900,อนิเมะแนะนำ,nokkyhuhu,14.0,0.0,RT yalovejong เทรดแนะนำ BL ทั้งฟิคและนิยายมังง...,2022-03-24 04:51:04+00:00,th,"Bangkok, Thailand",[],negative,66.67
4901,อนิเมะแนะนำ,ILYSB_I,33.0,0.0,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,2022-03-23 19:48:49+00:00,th,NaN,['แนะนำอ'],positive,66.67


In [54]:
df1 = dm.getrowwithkeys(['shounen']).sort_values('Keyword')
df1

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
2006,shounen,Shounen_w,0.0,1.0,@seongisdaddy never gonna happen ue,2022-03-19 08:17:18+00:00,NaN,NaN,NaN,NaN,NaN
24209,shounen,ipqiyv88,761.0,0.0,RT BroskeesNFTs BROSKEES x SHOUNEN To celebrat...,2022-03-25 13:01:30+00:00,en,NaN,[],neutral,0.00
24210,shounen,youtaroyoujo,15.0,0.0,RT RottingVale Had a very long and funny conve...,2022-03-25 05:22:26+00:00,en,NaN,[],positive,0.02
24211,shounen,DiegoHdzF,3778.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-25 07:24:29+00:00,en,Monterrey,[],neutral,0.00
24212,shounen,costa6770l,3587.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-25 06:13:29+00:00,en,NaN,[],neutral,0.00
...,...,...,...,...,...,...,...,...,...,...,...
20102,shounen,bellezzadellunv,2236.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24 16:26:52+00:00,en,ist,[],neutral,0.00
20103,shounen,ahhkeeh,2236.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24 16:26:56+00:00,en,NaN,[],neutral,0.00
20104,shounen,chiungwei17,2236.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24 16:27:04+00:00,en,NaN,[],neutral,0.00
20106,shounen,MENG5533,318.0,0.0,RT SekiraNFT SHOUNEN X SKIRAPleased to announc...,2022-03-24 16:27:21+00:00,en,johor,[],positive,0.50


In [55]:
dm.setnewDF(df1)
dm.getperiod('2022-03-21','2022-03-25')

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
2892,shounen,HipHopBigWin,6371.0,0.0,RT NFTSHOUNEN Playful perception of everyday o...,2022-03-21,en,tg- februarywinz,[],positive,0.333333
2893,shounen,0xpickawpickr,5740.0,0.0,RT NFTSHOUNEN Playful perception of everyday o...,2022-03-21,en,give away hunter （＾ω＾）,[],positive,0.333333
2894,shounen,pilihakucityx,5849.0,0.0,RT NFTSHOUNEN Playful perception of everyday o...,2022-03-21,en,"Central Java, Indonesia",[],positive,0.333333
2895,shounen,Shounen_w,8.0,0.0,RT THEMAzine Contributor signups are open for ...,2022-03-21,en,V O I D,[],positive,0.291667
2896,shounen,BnR_Kasep,5805.0,0.0,RT NFTSHOUNEN Playful perception of everyday o...,2022-03-21,en,"DKI Jakarta, Indonesia",[],positive,0.333333
...,...,...,...,...,...,...,...,...,...,...,...
22048,shounen,JassAnn6,422.0,0.0,RT WGMIIndustries WGMI Industries x SHOUNEN To...,2022-03-25,en,Twitter,[],neutral,0.000000
22071,shounen,Danieldew101,1263.0,0.0,RT BroskeesNFTs BROSKEES x SHOUNEN To celebrat...,2022-03-25,en,USA,[],neutral,0.000000
22099,shounen,CrickHelvey,1195.0,0.0,RT BroskeesNFTs BROSKEES x SHOUNEN To celebrat...,2022-03-25,en,thanh hóa,[],neutral,0.000000
22092,shounen,Bbbbbbssssss222,433.0,0.0,RT WGMIIndustries WGMI Industries x SHOUNEN To...,2022-03-25,en,NaN,[],neutral,0.000000


In [56]:
dm.setdefaultDF()

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
0,animation,dialhforhero,0.0,0.0,An animation trailer x100 times better than mo...,2022-03-19 11:44:27+00:00,NaN,NaN,NaN,NaN,NaN
1,animation,MaFt,0.0,0.0,@winstano My uncle @Billy_Allison did the anim...,2022-03-19 09:09:22+00:00,NaN,NaN,NaN,NaN,NaN
2,animation,plushladyy,0.0,0.0,Ghhh there was this animation about moon danci...,2022-03-19 09:39:41+00:00,NaN,NaN,NaN,NaN,NaN
3,animation,aeyvindr,0.0,0.0,I wonder how would the past me say seeing my d...,2022-03-19 12:14:58+00:00,NaN,NaN,NaN,NaN,NaN
4,animation,zeze_luva,0.0,0.0,"The plot of ""I’ve Gone Viral Thanks to My Crus...",2022-03-19 15:58:17+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4898,อนิเมะแนะนำ,Worra_K,41.0,0.0,RT bbrinkk มีกิจกรรมฟินๆมาแนะนำ สาววายถูกใจสิ่...,2022-03-24 10:49:48+00:00,th,NaN,['สาววายถ'],positive,97.30
4899,อนิเมะแนะนำ,W3NY7,33.0,0.0,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,2022-03-24 08:41:09+00:00,th,18 up /,['แนะนำอ'],positive,66.67
4900,อนิเมะแนะนำ,nokkyhuhu,14.0,0.0,RT yalovejong เทรดแนะนำ BL ทั้งฟิคและนิยายมังง...,2022-03-24 04:51:04+00:00,th,"Bangkok, Thailand",[],negative,66.67
4901,อนิเมะแนะนำ,ILYSB_I,33.0,0.0,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,2022-03-23 19:48:49+00:00,th,NaN,['แนะนำอ'],positive,66.67


In [7]:
for word in dm.collectword()[:10]:
    print(word)

KeyboardInterrupt: 

In [ ]:
keys = ['bl anime','anime comedy','anime romance','ต่างโลก','anime','animation','shounen','pixar',
        'harem','fantasy anime','sport anime','attack on titan','disney animation','animation studio',
        'shounen ai','shoujo','อนิเมะ','2d animation','อนิเมะแนะนำ','japan animation']
len(keys)

In [ ]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022_1.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
#df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d %m %Y')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
#mask = (df['Time']>='19-03-2022') & (df['Time']<='21-03-2022')
df.loc[mask]

In [ ]:
en_stops = set(stopwords.words('english'))
filename = ['tweet_data_2732022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1
        
dff = df.loc[df['Keyword'].isin(['pixar','อนิเมะ'])]
word = {}

for i in dff['Tweet']:
    if langdetect.detect(i) != 'th':
        allwords = i.split()
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    else:
        allwords = word_tokenize(i, engine='newmm')
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
del word['RT']
del word[' ']
sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
for i in sortword[:10]:
    print(i)

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

all_words = ['There', 'is', 'a', 'tree','near','the','river']
for word in all_words: 
    if word not in en_stops:
        print(word)

In [ ]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'สาขานี้พนักงานน่ารักให้บริการดี'
 
data = {'text':text}
 
headers = {
    'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"
    }
 
response = requests.post(url, data=data, headers=headers)
 
print(response.json())
response.json()['preprocess']['keyword']

In [ ]:
key = list(set(df['Keyword'].tolist()))